# API Server and API Object

In [ ]:
待補

# What is Pod

<img src='./img/pod.png'>

官方網站: https://kubernetes.io/docs/concepts/workloads/pods/

# Creating Pods

### Imperative 的方法: 使用鏡像

In [ ]:
kubectl run nginx --image=nginx 

In [ ]:
kubectl run client --image=busybox --command -- bin/sh -c 'sleep 100000'

### Declarative 的方法: 使用 YMAL 檔

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx-busybox
spec:
  containers:
  - name: nginx
    image: nginx
    ports:
    - containerPort: 80
  - name: busybox
    image: busybox
    command: ["/bin/sh"]
    args: ["-c", "while true; do echo hello; sleep 10;done"]

In [ ]:
kubectl apply -f nginx_busybox.yml

### 簡單的查詢 Pod 的 YAML 語法

In [ ]:
kubectl explain pods

In [ ]:
kubectl explain pod.spec | more

In [ ]:
kubectl explain pod.spec.containers | more

### kubectl 的 長選項 --dry-run

##### Server-side

In [ ]:
kubectl apply -f nginx_busybox.yml --dry-run=server

##### Client-side

In [ ]:
kubectl apply -f nginx.yml --dry-run=client
kubectl run web --image=nginx --dry-run=client -o yaml
kubectl run web --image=nginx --dry-run=client -o yaml > nginx.yml

### kubectl diff 的操作

In [ ]:
kubectl diff -f nginx_busybox.yml

<img src='./img/21.png'>

# Describing Pods

### 獲取 pod 列表

In [ ]:
kubectl get pods

In [ ]:
kubectl get pod nginx-busybox -o wide

In [ ]:
kubectl get pod nginx-busybox -o json

### 獲取 pod 的詳細訊息

In [ ]:
kubectl describe pod nginx-busybox

### API level 的 log

In [ ]:
kubectl get pod <pod-name> -v 6   # 或者 7,8,9 不同的 level，數值越大，得到的信息越詳细

  <img src='./img/1.png'>

In [ ]:
kubectl get pods -A -v 6 

<img src='./img/2.png'>

In [ ]:
kubectl get pods <pod-name> --watch -v 6

---

In [ ]:
kubectl get events --watch

### kubectl proxy 的使用

In [ ]:
kubectl get pods -v 6

In [ ]:
curl https://192.168.49.2:8443/api/v1/namespaces/default/pods?limit=500 --insecure

In [ ]:
kubectl proxy

In [ ]:
kubectl proxy &

In [ ]:
curl http://127.0.0.1:8001/api/v1/namespaces/default/pods?limit=500

In [ ]:
fg

# Geting into pods

In [ ]:
kubectl exec nginx-busybox -- date

In [ ]:
kubectl exec nginx-busybox -c busybox -- date

In [ ]:
查看容器 busybox 的域名。

In [ ]:
kubectl exec -it nginx-busybox -c busybox -- hostname -f

In [ ]:
kubectl exec -it nginx-busybox -c busybox -- sh 

In [ ]:
$ ip addr

In [ ]:
$ wget 127.0.0.1:80
$ more index.html

# Updating pods

### 已經有 yaml 文件

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx-busybox
spec:
  containers:
  - name: nginx
    image: nginx
    ports:
    - containerPort: 80
  - name: busybox
    image: busybox
    command: ["/bin/sh"]
    args: ["-c", "while true; do echo hello; sleep 10;done"]

In [ ]:
kubectl create -f nginx_busybox.yml
kubectl get pods

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx-busybox
spec:
  containers:
  - name: nginx
    image: nginx:1.22.0
    ports:
    - containerPort: 80
  - name: busybox
    image: busybox
    command: ["/bin/sh"]
    args: ["-c", "while true; do echo hello; sleep 10;done"]

In [ ]:
kubectl apply -f nginx_busybox.yml

### 沒有 yaml 文件: 產生一個

In [ ]:
kubectl run nginx --image nginx

In [ ]:
kubectl get pod nginx -o yaml > nginx.yml

### 沒有 yaml 文件: 使用 edit 命令

In [ ]:
kubectl edit pod nginx 

# Deleting  pods

In [ ]:
kubectl delete pod nginx

In [ ]:
kubectl delete -f nginx_busybox.yml

# Static Pod

### 什麼是 Static Pods

### 怎麼知道那些是 Static Pods

##### 方法一: 使用指令 kubectl describe

In [ ]:
kubectl get pods -A

In [ ]:
kubectl describe pod --namespace=kube-system etcd-minikube

<img src='./img/3.PNG'>

In [ ]:
kubectl describe pod --namespace=kube-system storage-provisioner

<img src='./img/4.png'>

##### 方法二: 去找 static pod 的預設配置文件

In [ ]:
cat /var/lib/kubelet/config.yaml 

<img src='./img/5.png'>

In [ ]:
ls /etc/kubernetes/manifests

<img src='./img/6.png'>

### 創建 static pods

In [ ]:
kubectl run web --image=nginx --dry-run=client -o yaml > web.yaml
kubectl get pods

### 怎麼知道 k8s 的 config.yaml 文件放哪

In [ ]:
ps aux | grep kubelet

<img src='./img/7.png'>

# Init containers

參考官方文檔: https://kubernetes.io/docs/concepts/workloads/pods/init-containers/

<img src='./img/9.PNG'>

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: pod-with-init-containers
spec:
  initContainers:
  - name: init-service
    image: busybox
    command: ["sh", "-c", "echo waiting for sercice; sleep 2"]
  - name: init-database
    image: busybox
    command: ["sh", "-c", "echo waiting for database; sleep 2"]
  containers:
  - name: app-container
    image: nginx

In [ ]:
kubectl apply -f init_container.yaml

In [ ]:
kubectl get pod --watch

In [ ]:
kubectl describe pod pod-with-init-containers

# Pod Lifecycle

參考官方文檔: https://kubernetes.io/docs/concepts/workloads/pods/pod-lifecycle/

### SIGTERM vs SIGKILL: What’s the Difference

參考文章: https://linuxhandbook.com/sigterm-vs-sigkill/

<img src='./img/10.png'>

### pod phase

<img src='./img/11.png'>

### Container state

### Stopping/Terminating Pods

In [ ]:
kubectl delete pod podName

In [ ]:
kubectl delete pod <name> --grace-period=<seconds>

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: web
spec:
  terminationGracePeriodSeconds: 10
  containers:
  - image: nginx
    name: web

In [ ]:
kubectl delete pod podName --grace-period=0 --force

### Persistency of Pod

### Container Restart Policy

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: null
  labels:
    run: web
  name: web
spec:
  containers:
  - image: nginx
    name: web
    resources: {}
  dnsPolicy: ClusterFirst
  restartPolicy: Always

### 範例

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: busybox
spec:
  containers:
  - name: busybox
    image: busybox
    command: ["/bin/sh"]
    args: ["-c", "while true; do echo hello; sleep 10000;done"]